In [1]:
import util.utils as utils

percentage = 0.02
rand_seed = 42
data_sampling_freq = '10s'
rolling_window_size = '10s'

df_result = utils.get_binary(percentage, rand_seed)
df_face = utils.get_face(percentage, rand_seed, data_sampling_freq, rolling_window_size)

display(df_face)
display(df_result)

CLNFgaze_x_0  CLNFgaze_y_0  CLNFgaze_z_0  CLNFgaze_x_1  \
ID  TIMESTAMP                                                                 
386 0 days 00:00:00      0.020882      0.114932     -0.985159     -0.187369   
    0 days 00:00:10      0.060876      0.158891     -0.979911     -0.185456   
    0 days 00:00:20      0.048942      0.159756     -0.984069     -0.185223   
    0 days 00:00:30      0.051738      0.165190     -0.983668     -0.174363   
    0 days 00:00:40      0.049782      0.190049     -0.979635     -0.177583   
...                           ...           ...           ...           ...   
391 0 days 00:10:40     -0.000396      0.162813     -0.982788     -0.172738   
    0 days 00:10:50     -0.002969      0.168443     -0.975233     -0.173063   
    0 days 00:11:00     -0.008404      0.096919     -0.986678     -0.171271   
    0 days 00:11:10     -0.009584      0.035722     -0.991693     -0.076398   
    0 days 00:11:20      0.000000      0.000000     -1.000000      0.000000   

                     CLNFgaze_y_1  CLNFgaze_z_1  CLNFgaze_x_h0  CLNFgaze_y_h0  \
ID  TIMESTAMP                                                                   
386 0 days 00:00:00      0.090528     -0.967401       0.113978       0.211860   
    0 days 00:00:10      0.139311     -0.964433       0.013874       0.204188   
    0 days 00:00:20      0.131315     -0.971070       0.017197       0.242405   
    0 days 00:00:30      0.125572     -0.974557       0.018452       0.232494   
    0 days 00:00:40      0.144044     -0.972055       0.019017       0.253643   
...                           ...           ...            ...            ...   
391 0 days 00:10:40      0.182752     -0.962403       0.132738       0.410371   
    0 days 00:10:50      0.101383     -0.971495       0.400424       0.394142   
    0 days 00:11:00      0.141179     -0.946717       0.234962       0.192641   
    0 days 00:11:10      0.035174     -0.982470       0.060823       0.138469   
    0 days 00:11:20      0.000000     -1.000000       0.000000       0.000000   

                     CLNFgaze_z_h0  CLNFgaze_x_h1  ...  CLNFfeatures3D_Z58  \
ID  TIMESTAMP                                      ...                       
386 0 days 00:00:00      -0.946309      -0.091666  ...          519.613237   
    0 days 00:00:10      -0.971987      -0.230672  ...          540.856410   
    0 days 00:00:20      -0.964628      -0.215217  ...          519.459970   
    0 days 00:00:30      -0.970581      -0.206811  ...          517.561633   
    0 days 00:00:40      -0.966128      -0.207692  ...          520.462397   
...                            ...            ...  ...                 ...   
391 0 days 00:10:40      -0.886184      -0.039702  ...          596.082983   
    0 days 00:10:50      -0.777415       0.270857  ...          611.819750   
    0 days 00:11:00      -0.824262       0.089705  ...          549.835507   
    0 days 00:11:10      -0.941188      -0.003228  ...          642.796860   
    0 days 00:11:20      -1.000000       0.000000  ...          468.863000   

                     CLNFfeatures3D_Z59  CLNFfeatures3D_Z60  \
ID  TIMESTAMP                                                 
386 0 days 00:00:00          523.152257          528.202373   
    0 days 00:00:10          546.058773          553.371500   
    0 days 00:00:20          525.978237          534.036180   
    0 days 00:00:30          524.068173          531.824040   
    0 days 00:00:40          527.595803          536.371060   
...                                 ...                 ...   
391 0 days 00:10:40          600.297053          605.735340   
    0 days 00:10:50          611.001257          612.771850   
    0 days 00:11:00          551.907377          558.196050   
    0 days 00:11:10          642.991900          645.226560   
    0 days 00:11:20          466.043000          460.653000   

                     CLNFfeatures3D_Z61  CLNFfeatures3D_Z62  \
ID  TIMESTAMP                            

,PHQ_Binary
ID,
386,1
391,0


### EDA

In [2]:
# eigenfaces 
# Display all column names in df_face

Columns containing 'hog':
['CLNFhog_feature_0', 'CLNFhog_feature_1', 'CLNFhog_feature_2', 'CLNFhog_feature_3', 'CLNFhog_feature_4', 'CLNFhog_feature_5', 'CLNFhog_feature_6', 'CLNFhog_feature_7', 'CLNFhog_feature_8', 'CLNFhog_feature_9', 'CLNFhog_feature_10', 'CLNFhog_feature_11', 'CLNFhog_feature_12', 'CLNFhog_feature_13', 'CLNFhog_feature_14', 'CLNFhog_feature_15', 'CLNFhog_feature_16', 'CLNFhog_feature_17', 'CLNFhog_feature_18', 'CLNFhog_feature_19', 'CLNFhog_feature_20', 'CLNFhog_feature_21', 'CLNFhog_feature_22', 'CLNFhog_feature_23', 'CLNFhog_feature_24', 'CLNFhog_feature_25', 'CLNFhog_feature_26', 'CLNFhog_feature_27', 'CLNFhog_feature_28', 'CLNFhog_feature_29', 'CLNFhog_feature_30', 'CLNFhog_feature_31', 'CLNFhog_feature_32', 'CLNFhog_feature_33', 'CLNFhog_feature_34', 'CLNFhog_feature_35', 'CLNFhog_feature_36', 'CLNFhog_feature_37', 'CLNFhog_feature_38', 'CLNFhog_feature_39', 'CLNFhog_feature_40', 'CLNFhog_feature_41', 'CLNFhog_feature_42', 'CLNFhog_feature_43', 'CLNFhog_featur

In [ ]:
# distribution of landmarks / action units


### train test split

In [ ]:
# this is just wring and weird...
import pandas as pd

df = pd.merge(df_face, df_result, on='ID')

window_size = 5  # Adjust the window size as needed
features = ['CLNFgaze_x_0', 'CLNFgaze_y_0', 'CLNFgaze_z_0', 'CLNFgaze_x_1', 'CLNFgaze_y_1', 'CLNFgaze_z_1',
            'CLNFgaze_x_h0', 'CLNFgaze_y_h0', 'CLNFgaze_z_h0', 'CLNFgaze_x_h1', 'CLNFgaze_y_h1', 'CLNFgaze_z_h1',
            'CLNFAUs_AU01_r', 'CLNFAUs_AU02_r', 'CLNFAUs_AU04_r', 'CLNFAUs_AU05_r', 'CLNFAUs_AU06_r', 'CLNFAUs_AU09_r',
            'CLNFAUs_AU10_r', 'CLNFAUs_AU12_r', 'CLNFAUs_AU14_r', 'CLNFAUs_AU15_r', 'CLNFAUs_AU17_r', 'CLNFAUs_AU20_r',
            'CLNFAUs_AU25_r', 'CLNFAUs_AU26_r', 'CLNFAUs_AU04_c', 'CLNFAUs_AU12_c', 'CLNFAUs_AU15_c', 'CLNFAUs_AU23_c',
            'CLNFAUs_AU28_c', 'CLNFAUs_AU45_c', 'CLNFfeatures_x0', 'CLNFfeatures_x1', 'CLNFfeatures_x2',
            'CLNFfeatures_x3', 'CLNFfeatures_x4', 'CLNFfeatures_x5', 'CLNFfeatures_x6', 'CLNFfeatures_x7',
            'CLNFfeatures_x8', 'CLNFfeatures_x9', 'CLNFfeatures_x10', 'CLNFfeatures_x11', 'CLNFfeatures_x12',
            'CLNFfeatures_x13', 'CLNFfeatures_x14', 'CLNFfeatures_x15', 'CLNFfeatures_x16', 'CLNFfeatures_x17',
            'CLNFfeatures_x18', 'CLNFfeatures_x19', 'CLNFfeatures_x20', 'CLNFfeatures_x21', 'CLNFfeatures_x22',
            'CLNFfeatures_x23', 'CLNFfeatures_x24', 'CLNFfeatures_x25', 'CLNFfeatures_x26', 'CLNFfeatures_x27',
            'CLNFfeatures_x28', 'CLNFfeatures_x29', 'CLNFfeatures_x30', 'CLNFfeatures_x31', 'CLNFfeatures_x32',
            'CLNFfeatures_x33', 'CLNFfeatures_x34', 'CLNFfeatures_x35', 'CLNFfeatures_x36', 'CLNFfeatures_x37',
            'CLNFfeatures_x38', 'CLNFfeatures_x39', 'CLNFfeatures_x40', 'CLNFfeatures_x41', 'CLNFfeatures_x42',
            'CLNFfeatures_x43', 'CLNFfeatures_x44', 'CLNFfeatures_x45', 'CLNFfeatures_x46', 'CLNFfeatures_x47',
            'CLNFfeatures_x48', 'CLNFfeatures_x49', 'CLNFfeatures_x50', 'CLNFfeatures_x51', 'CLNFfeatures_x52',
            'CLNFfeatures_x53', 'CLNFfeatures_x54', 'CLNFfeatures_x55', 'CLNFfeatures_x56', 'CLNFfeatures_x57',
            'CLNFfeatures_x58', 'CLNFfeatures_x59', 'CLNFfeatures_x60', 'CLNFfeatures_x61', 'CLNFfeatures_x62',
            'CLNFfeatures_x63', 'CLNFfeatures_x64', 'CLNFfeatures_x65', 'CLNFfeatures_x66', 'CLNFfeatures_x67',
            'CLNFfeatures_y0', 'CLNFfeatures_y1', 'CLNFfeatures_y2', 'CLNFfeatures_y3', 'CLNFfeatures_y4',
            'CLNFfeatures_y5', 'CLNFfeatures_y6', 'CLNFfeatures_y7', 'CLNFfeatures_y8', 'CLNFfeatures_y9',
            'CLNFfeatures_y10', 'CLNFfeatures_y11', 'CLNFfeatures_y12', 'CLNFfeatures_y13', 'CLNFfeatures_y14',
            'CLNFfeatures_y15', 'CLNFfeatures_y16', 'CLNFfeatures_y17', 'CLNFfeatures_y18', 'CLNFfeatures_y19',
            'CLNFfeatures_y20', 'CLNFfeatures_y21', 'CLNFfeatures_y22', 'CLNFfeatures_y23', 'CLNFfeatures_y24',
            'CLNFfeatures_y25', 'CLNFfeatures_y26', 'CLNFfeatures_y27', 'CLNFfeatures_y28', 'CLNFfeatures_y29',
            'CLNFfeatures_y30', 'CLNFfeatures_y31', 'CLNFfeatures_y32', 'CLNFfeatures_y33', 'CLNFfeatures_y34',
            'CLNFfeatures_y35', 'CLNFfeatures_y36', 'CLNFfeatures_y37', 'CLNFfeatures_y38', 'CLNFfeatures_y39',
            'CLNFfeatures_y40', 'CLNFfeatures_y41', 'CLNFfeatures_y42', 'CLNFfeatures_y43', 'CLNFfeatures_y44',
            'CLNFfeatures_y45', 'CLNFfeatures_y46', 'CLNFfeatures_y47', 'CLNFfeatures_y48', 'CLNFfeatures_y49',
            'CLNFfeatures_y50', 'CLNFfeatures_y51', 'CLNFfeatures_y52', 'CLNFfeatures_y53', 'CLNFfeatures_y54',
            'CLNFfeatures_y55', 'CLNFfeatures_y56', 'CLNFfeatures_y57', 'CLNFfeatures_y58', 'CLNFfeatures_y59',
            'CLNFfeatures_y60', 'CLNFfeatures_y61', 'CLNFfeatures_y62', 'CLNFfeatures_y63', 'CLNFfeatures_y64',
            'CLNFfeatures_y65', 'CLNFfeatures_y66', 'CLNFfeatures_y67', 'CLNFpose_Tx', 'CLNFpose_Ty', 'CLNFpose_Tz',
            'CLNFpose_Rx', 'CLNFpose_Ry', 'CLNFpose_Rz', 'CLNFfeatures3D_X0', 'CLNFfeatures3D_X1', 'CLNFfeatures3D_X2',
            'CLNFfeatures3D_X3', 'CLNFfeatures3D_X4', 'CLNFfeatures3D_X5', 'CLNFfeatures3D_X6', 'CLNFfeatures3D_X7',
            'CLNFfeatures3D_X8', 'CLNFfeatures3D_X9', 'CLNFfeatures3D_X10', 'CLNFfeatures3D_X11', 'CLNFfeatures3D_X12',
            'CLNFfeatures3D_X13', 'CLNFfeatures3D_X14', 'CLNFfeatures3D_X15', 'CLNFfeatures3D_X16',
            'CLNFfeatures3D_X17', 'CLNFfeatures3D_X18', 'CLNFfeatures3D_X19', 'CLNFfeatures3D_X20',
            'CLNFfeatures3D_X21', 'CLNFfeatures3D_X22', 'CLNFfeatures3D_X23', 'CLNFfeatures3D_X24',
            'CLNFfeatures3D_X25', 'CLNFfeatures3D_X26', 'CLNFfeatures3D_X27', 'CLNFfeatures3D_X28',
            'CLNFfeatures3D_X29', 'CLNFfeatures3D_X30', 'CLNFfeatures3D_X31', 'CLNFfeatures3D_X32',
            'CLNFfeatures3D_X33', 'CLNFfeatures3D_X34', 'CLNFfeatures3D_X35', 'CLNFfeatures3D_X36',
            'CLNFfeatures3D_X37', 'CLNFfeatures3D_X38', 'CLNFfeatures3D_X39', 'CLNFfeatures3D_X40',
            'CLNFfeatures3D_X41', 'CLNFfeatures3D_X42', 'CLNFfeatures3D_X43', 'CLNFfeatures3D_X44',
            'CLNFfeatures3D_X45', 'CLNFfeatures3D_X46', 'CLNFfeatures3D_X47', 'CLNFfeatures3D_X48',
            'CLNFfeatures3D_X49', 'CLNFfeatures3D_X50', 'CLNFfeatures3D_X51', 'CLNFfeatures3D_X52',
            'CLNFfeatures3D_X53', 'CLNFfeatures3D_X54', 'CLNFfeatures3D_X55', 'CLNFfeatures3D_X56',
            'CLNFfeatures3D_X57', 'CLNFfeatures3D_X58', 'CLNFfeatures3D_X59', 'CLNFfeatures3D_X60',
            'CLNFfeatures3D_X61', 'CLNFfeatures3D_X62', 'CLNFfeatures3D_X63', 'CLNFfeatures3D_X64',
            'CLNFfeatures3D_X65', 'CLNFfeatures3D_X66', 'CLNFfeatures3D_X67', 'CLNFfeatures3D_Y0', 'CLNFfeatures3D_Y1',
            'CLNFfeatures3D_Y2', 'CLNFfeatures3D_Y3', 'CLNFfeatures3D_Y4', 'CLNFfeatures3D_Y5', 'CLNFfeatures3D_Y6',
            'CLNFfeatures3D_Y7', 'CLNFfeatures3D_Y8', 'CLNFfeatures3D_Y9', 'CLNFfeatures3D_Y10', 'CLNFfeatures3D_Y11',
            'CLNFfeatures3D_Y12', 'CLNFfeatures3D_Y13', 'CLNFfeatures3D_Y14', 'CLNFfeatures3D_Y15',
            'CLNFfeatures3D_Y16', 'CLNFfeatures3D_Y17', 'CLNFfeatures3D_Y18', 'CLNFfeatures3D_Y19',
            'CLNFfeatures3D_Y20', 'CLNFfeatures3D_Y21', 'CLNFfeatures3D_Y22', 'CLNFfeatures3D_Y23',
            'CLNFfeatures3D_Y24', 'CLNFfeatures3D_Y25', 'CLNFfeatures3D_Y26', 'CLNFfeatures3D_Y27',
            'CLNFfeatures3D_Y28', 'CLNFfeatures3D_Y29', 'CLNFfeatures3D_Y30', 'CLNFfeatures3D_Y31',
            'CLNFfeatures3D_Y32', 'CLNFfeatures3D_Y33', 'CLNFfeatures3D_Y34', 'CLNFfeatures3D_Y35',
            'CLNFfeatures3D_Y36', 'CLNFfeatures3D_Y37', 'CLNFfeatures3D_Y38', 'CLNFfeatures3D_Y39',
            'CLNFfeatures3D_Y40', 'CLNFfeatures3D_Y41', 'CLNFfeatures3D_Y42', 'CLNFfeatures3D_Y43',
            'CLNFfeatures3D_Y44', 'CLNFfeatures3D_Y45', 'CLNFfeatures3D_Y46', 'CLNFfeatures3D_Y47',
            'CLNFfeatures3D_Y48', 'CLNFfeatures3D_Y49', 'CLNFfeatures3D_Y50', 'CLNFfeatures3D_Y51',
            'CLNFfeatures3D_Y52', 'CLNFfeatures3D_Y53', 'CLNFfeatures3D_Y54', 'CLNFfeatures3D_Y55',
            'CLNFfeatures3D_Y56', 'CLNFfeatures3D_Y57', 'CLNFfeatures3D_Y58', 'CLNFfeatures3D_Y59',
            'CLNFfeatures3D_Y60', 'CLNFfeatures3D_Y61', 'CLNFfeatures3D_Y62', 'CLNFfeatures3D_Y63',
            'CLNFfeatures3D_Y64', 'CLNFfeatures3D_Y65', 'CLNFfeatures3D_Y66', 'CLNFfeatures3D_Y67', 'CLNFfeatures3D_Z0',
            'CLNFfeatures3D_Z1', 'CLNFfeatures3D_Z2', 'CLNFfeatures3D_Z3', 'CLNFfeatures3D_Z4', 'CLNFfeatures3D_Z5',
            'CLNFfeatures3D_Z6', 'CLNFfeatures3D_Z7', 'CLNFfeatures3D_Z8', 'CLNFfeatures3D_Z9', 'CLNFfeatures3D_Z10',
            'CLNFfeatures3D_Z11', 'CLNFfeatures3D_Z12', 'CLNFfeatures3D_Z13', 'CLNFfeatures3D_Z14',
            'CLNFfeatures3D_Z15', 'CLNFfeatures3D_Z16', 'CLNFfeatures3D_Z17', 'CLNFfeatures3D_Z18',
            'CLNFfeatures3D_Z19', 'CLNFfeatures3D_Z20', 'CLNFfeatures3D_Z21', 'CLNFfeatures3D_Z22',
            'CLNFfeatures3D_Z23', 'CLNFfeatures3D_Z24', 'CLNFfeatures3D_Z25', 'CLNFfeatures3D_Z26',
            'CLNFfeatures3D_Z27', 'CLNFfeatures3D_Z28', 'CLNFfeatures3D_Z29', 'CLNFfeatures3D_Z30',
            'CLNFfeatures3D_Z31', 'CLNFfeatures3D_Z32', 'CLNFfeatures3D_Z33', 'CLNFfeatures3D_Z34',
            'CLNFfeatures3D_Z35', 'CLNFfeatures3D_Z36', 'CLNFfeatures3D_Z37', 'CLNFfeatures3D_Z38',
            'CLNFfeatures3D_Z39', 'CLNFfeatures3D_Z40', 'CLNFfeatures3D_Z41', 'CLNFfeatures3D_Z42',
            'CLNFfeatures3D_Z43', 'CLNFfeatures3D_Z44', 'CLNFfeatures3D_Z45', 'CLNFfeatures3D_Z46',
            'CLNFfeatures3D_Z47', 'CLNFfeatures3D_Z48', 'CLNFfeatures3D_Z49', 'CLNFfeatures3D_Z50',
            'CLNFfeatures3D_Z51', 'CLNFfeatures3D_Z52', 'CLNFfeatures3D_Z53', 'CLNFfeatures3D_Z54',
            'CLNFfeatures3D_Z55', 'CLNFfeatures3D_Z56', 'CLNFfeatures3D_Z57', 'CLNFfeatures3D_Z58',
            'CLNFfeatures3D_Z59', 'CLNFfeatures3D_Z60', 'CLNFfeatures3D_Z61', 'CLNFfeatures3D_Z62',
            'CLNFfeatures3D_Z63', 'CLNFfeatures3D_Z64', 'CLNFfeatures3D_Z65', 'CLNFfeatures3D_Z66',
            'CLNFfeatures3D_Z67']

# Apply sliding window
sliding_features = (
    df
    .groupby('ID')[features]
    .rolling(window_size)
    .mean()
    .reset_index(level=0, drop=True)
)

# Append rolling features back to merged dataframe and drop any NaNs
df = df.join(sliding_features, rsuffix='_rolling').dropna()

display(df)

In [ ]:
# train test split

from sklearn.model_selection import TimeSeriesSplit

X = df[[f'{feature}_rolling' for feature in features]]  # Rolling features
y = df['PHQ_Binary']

# Use TimeSeriesSplit for splitting
tscv = TimeSeriesSplit(n_splits=5)

### pipeline & hyperparameter tuning

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# creating pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', RandomForestRegressor())
])

# hyperparameter tuning
# Define parameter grid for tuning
param_grid = {
    'model__n_estimators': [50, 100, 200],
    'model__max_depth': [10, 20, None],
    'model__min_samples_split': [2, 5, 10]
}

# Use GridSearchCV with TimeSeriesSplit cross-validation
grid_search = GridSearchCV(
    pipeline, param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs=-1
)

# Fit grid search to the data
grid_search.fit(X, y)

### evaluation

In [2]:
# from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
# 
# y_pred = grid_search.predict(X_test)
# 
# y_test = y_test.to_numpy()
# # Debug: Check the shapes and types of y_pred
# print(f'y_pred: {y_pred}')
# print(f'y_test: {y_test}')
# 
# # Generate and display classification report
# print("Classification Report:")
# print(classification_report(y_test, y_pred))
# 
# # Generate and display confusion matrix
# cm = confusion_matrix(y_test, y_pred, labels=grid_search.classes_)
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=grid_search.classes_)
# disp.plot(cmap=plt.cm.Blues)
# plt.show()
# 
